In [1]:
SF = 100
ws = 'sqlengines'
lh ='benchmarks'

In [2]:
!pip install -q duckdb     --upgrade
!pip install -q deltalake  --upgrade

# Install Package

In [3]:
!pip show deltalake
!pip show duckdb

Name: deltalake
Version: 0.22.3
Summary: Native Delta Lake Python binding based on delta-rs with Pandas integration
Home-page: https://github.com/delta-io/delta-rs
Author: Qingping Hou <dave2008713@gmail.com>, Will Jones <willjones127@gmail.com>
Author-email: Qingping Hou <dave2008713@gmail.com>, Will Jones <willjones127@gmail.com>
License: Apache-2.0
Location: /home/trusted-service-user/jupyter-env/python3.11/lib/python3.11/site-packages
Requires: pyarrow
Required-by: 
Name: duckdb
Version: 1.1.3
Summary: DuckDB in-process database
Home-page: https://www.duckdb.org
Author: 
Author-email: 
License: MIT
Location: /home/trusted-service-user/jupyter-env/python3.11/lib/python3.11/site-packages
Requires: 
Required-by: 


# Import Package

In [4]:
from   psutil import *
import pandas as pd
import duckdb
import pathlib
from   deltalake import convert_to_deltalake , DeltaTable
from   deltalake.writer import write_deltalake
import time
from   datetime import datetime

In [5]:
core=cpu_count()
core

2

# Generate Delta

In [6]:
# you can keep lineitem only for this test
# you need to mound a lakehouse, duckdb can't write directly to abfss path, reads works fine since 1.1.3
#table_list = ['nation','region','customer','supplier','lineitem','orders','partsupp','part']
table_list = ['lineitem']

In [7]:
%%time
Table_Path = '/lakehouse/default/Tables'
Files_Path = '/lakehouse/default/Files'
if not pathlib.Path(f'{Table_Path}/CH{SF:02d}').exists():
  if not pathlib.Path(f'{Files_Path}/tpch_{SF:02d}.duckdb').exists():
    print('generate new database')
    pathlib.Path(f'{Files_Path}').mkdir(parents=True, exist_ok=True)
    con=duckdb.connect(f'{Files_Path}/tpch_{SF:02d}.duckdb')
    con.sql(f"CALL dbgen(sf={SF}) ")
  else:
     con=duckdb.connect(f'{Files_Path}/tpch_{SF:02d}.duckdb',read_only=1)
     print('DB exist')
  for tbl in table_list:
      pathlib.Path(f'{Table_Path}/CH{SF:02d}/{tbl}').mkdir(parents=True, exist_ok=True)
      con.sql(f"""
      COPY (SELECT * FROM {tbl}) TO '{Table_Path}/CH{SF:02d}/{tbl}' (FORMAT PARQUET,PER_THREAD_OUTPUT TRUE)
       """)
      convert_to_deltalake(f'{Table_Path}/CH{SF:02d}/{tbl}',storage_options={"allow_unsafe_rename":"true"})
  con.close()
else:
  print("Already Generated")

Already Generated
CPU times: user 940 µs, sys: 155 µs, total: 1.09 ms
Wall time: 213 ms


# Delta size function

In [8]:
# @title
#the function read all the tables in the default Lakehouse and save it in a Table Delta_stats, change the url to reflect your destination
def get_stats(Source,Destination):
    appended_data = []
    List_tables = [ f.name for f in os.scandir(Source) if f.is_dir() ]
    stat_table = os.path.basename(os.path.normpath(Destination))
    if stat_table in List_tables: List_tables.remove(stat_table)
    for tbl in List_tables :
        try:
          dt = DeltaTable(Source+ tbl)
          xx= dt.get_add_actions(flatten=True).to_pandas()
          if 'tags.VORDER' in xx.columns:
               vorder =True
          else :
               vorder =False 
          deltalog=pd.DataFrame([[tbl,xx['size_bytes'].sum(),vorder]],columns=['tbl','size_bytes','vorder'])
          delta = [Source+tbl + "/" + i for i in dt.files()]
          duckdb.sql(f'from parquet_scan({delta})').to_view(tbl)
          print(tbl)
          df= duckdb.sql(f'''
          with x as (
          select file_name,num_rows,num_row_groups, '{tbl}' as tbl from parquet_file_metadata({delta})
          )
          select x.tbl, file_name,num_rows,num_row_groups, ceil(size_bytes/(1024*1024)) as size,vorder from x
          left join deltalog as y
          on x.tbl=y.tbl
          ''').df()
          appended_data.append(df)
        except  Exception as er:
              print(f"this table is not Delta :{tbl}") 
    appended_data = pd.concat(appended_data,ignore_index=True)
    appended_data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    write_deltalake(Destination,appended_data,mode="overwrite",storage_options={"allow_unsafe_rename":"true"})

    dt = DeltaTable(Destination)
    delta = [Destination + "/" + i for i in dt.files()]
    duckdb.sql(f'from parquet_scan({delta})').to_view("xxx")

    duckdb.sql('''
    select tbl,
    sum(num_rows) as total_rows,
    count(*) as num_files,
    list(num_rows) as rows_per_file,
    sum(num_row_groups) as num_row_group,
    cast(ceil(total_rows/num_row_group) as integer) as Average_row_group,
    min(size) as file_size_MB,
    vorder
    from xxx
    group by all
    order by file_size_MB ASC
    ''').show(max_width=150)


# Run test

In [9]:
%%time
duckdb.sql(f"""  CREATE or replace SECRET onelake (TYPE AZURE,PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{notebookutils.credentials.getToken('storage')}')  """)
df = duckdb.sql(f''' FROM delta_scan('abfss://{ws}@onelake.dfs.fabric.microsoft.com/{lh}.Lakehouse/Tables/CH{SF:02d}/lineitem') ''').record_batch()
write_deltalake(f'abfss://{ws}@onelake.dfs.fabric.microsoft.com/{lh}.Lakehouse/Tables/dbo/duckdb',
                                            df,
                                            max_rows_per_group  = 16_000_000,
                                            min_rows_per_group  =  8_000_000,
                                            max_rows_per_file   = 48_000_000,
                                            mode="overwrite",
                                            engine='pyarrow') 

<timed exec>:3: DeprecationWarning: pyarrow engine is deprecated and will be removed in v1.0


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 17min 4s, sys: 1min 46s, total: 18min 50s
Wall time: 21min 30s


In [10]:
get_stats(f"/lakehouse/default/Tables/dbo/","/lakehouse/default/Tables/dbo/xx_deltastats")

duckdb


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

polars


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

result
┌─────────┬────────────┬───────────┬────────────────────────────────────────────────────┬───────────────┬───────────────────┬──────────────┬─────────┐
│   tbl   │ total_rows │ num_files │                   rows_per_file                    │ num_row_group │ Average_row_group │ file_size_MB │ vorder  │
│ varchar │   int128   │   int64   │                      int64[]                       │    int128     │       int32       │    double    │ boolean │
├─────────┼────────────┼───────────┼────────────────────────────────────────────────────┼───────────────┼───────────────────┼──────────────┼─────────┤
│ result  │     121642 │         4 │ [88, 99, 99, 121356]                               │             4 │             30411 │          1.0 │ false   │
│ polars  │   50000000 │        18 │ [2807096, 2806312, 2806312, 2806312, 2806312, 28…  │            54 │            925926 │       1869.0 │ false   │
│ duckdb  │  600037902 │        13 │ [48000000, 48000000, 48000000, 48000000, 4800000… 